<a href="https://colab.research.google.com/github/maiatechdev/Consult_Embeddings/blob/main/Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install SDK
! pip install -U -q google-generativeai

In [31]:
# Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY="DIGITE_SUA_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
#Listando
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#Exemplo de embedding
title = "A proxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A proxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo: \n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessivel de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.055287853, -0.045071978, -0.016810164, 0.023383114, 0.062377244, -0.0058830245, -0.023487896, -0.024254458, 0.060670007, 0.057492994, 0.010749352, 0.010297517, -0.049809724, -0.021173771, 0.005105977, -0.02191265, 0.02349955, -0.0054176147, -0.03675188, 0.00016244859, 0.010119578, 0.007016564, -0.032402832, -0.078781866, -0.012819149, 0.024697518, 0.019796731, -0.03518817, -0.0431032, 0.030054959, -0.050483525, 0.048561215, -0.033703174, 0.014331133, -0.0311835, -0.044920366, -0.019316599, -0.044040196, 0.00069595623, -0.0035827379, 0.010806096, -0.08516999, -0.014373191, 0.017869657, -0.009944237, -0.019142, 0.045489073, 0.04826275, 0.01934027, -0.05754051, 0.032133747, 0.021986295, 0.066533595, -0.044725522, 0.0012717796, -0.009182861, 0.024347227, -0.02769624, 0.023918012, 0.0055471277, -0.00543148, 0.026411198, -0.014193929, 0.0572946, 0.025930459, -0.057717614, -0.04313574, 0.0008182028, 0.010660168, 0.037762858, 0.0025382927, 0.0061158277, 0.058925334, -0.0627985

In [17]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Materialismo Histórico Dialético",
    "Conteúdo": "O Materialismo Histórico Dialético, elaborado por Karl Marx e Friedrich Engels, é uma lente poderosa para analisar a sociedade e sua transformação ao longo do tempo. Ele se baseia em dois pilares: Materialismo Histórico e Dialético"}

DOCUMENT2 = {
    "Título": "Indústria Musical",
    "Conteúdo": "Taylor Swift, artista pop de renome internacional, ascendeu ao estrelato em uma era de profunda transformação na indústria musical. A ascensão da internet e das plataformas digitais reconfigurou o cenário, gerando novas oportunidades e desafios. Nesse contexto, Swift se destaca por sua perspicácia em navegar pelas mudanças e construir uma carreira autônoma."}

DOCUMENT3 = {
    "Título": "Skateboarding e sua Cultura",
    "Conteúdo": "O skateboarding transcende a simples atividade física, consolidando-se como um esporte, uma arte, uma forma de expressão e até mesmo um meio de transporte. Mais do que deslizar sobre um shape e rodas, o skate propõe a superação de limites, a busca por novos desafios e a constante evolução individual. Seus praticantes, os skatistas, encontram na modalidade um espaço para criatividade, liberdade e conexão com a comunidade."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [19]:
#Convertendo os documentos em dataframe e definindo colunas
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Materialismo Histórico Dialético,"O Materialismo Histórico Dialético, elaborado ..."
1,Indústria Musical,"Taylor Swift, artista pop de renome internacio..."
2,Skateboarding e sua Cultura,O skateboarding transcende a simples atividade...


In [8]:
model = "models/embedding-001"

In [20]:
#Função para filtrar embeddings
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [21]:
#Criando a coluna de Embeddings(vetores)
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Materialismo Histórico Dialético,"O Materialismo Histórico Dialético, elaborado ...","[0.026480436, -0.0511245, -0.059286173, 0.0451..."
1,Indústria Musical,"Taylor Swift, artista pop de renome internacio...","[0.032883577, -0.06898503, 0.041098308, 0.0616..."
2,Skateboarding e sua Cultura,O skateboarding transcende a simples atividade...,"[-0.0048513804, -0.015569729, 0.0039143306, 0...."


In [22]:
#Consultando por semântica
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [23]:
consulta = "Qual esporte posso praticar para melhorar meu desenvolvimento cognitivo?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O skateboarding transcende a simples atividade física, consolidando-se como um esporte, uma arte, uma forma de expressão e até mesmo um meio de transporte. Mais do que deslizar sobre um shape e rodas, o skate propõe a superação de limites, a busca por novos desafios e a constante evolução individual. Seus praticantes, os skatistas, encontram na modalidade um espaço para criatividade, liberdade e conexão com a comunidade.


In [24]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [25]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

O skate não é só um rolê, é muito mais! É um esporte, uma arte, um jeito de se expressar e até um meio de transporte. Não é só deslizar no shape, é superar limites, buscar desafios e evoluir sempre. Os skatistas encontram no skate um lugar pra soltar a criatividade, ser livre e se conectar com a galera.
